# Practica 4

## Ejercicio 1

### a

Queremos solucionar el problema de los case sensitive usando comparadores, esto significa modificar las funciones shuffle y sort. La salida de los mappers es <first_letter, 1> lo que nosotros tenemos que hacer es solucionar que "A" y "a" vayan al mismo nodo y se ordenen juntas, podemos modificar los cmp usando alguna función estilo to lower_case(). 

In [2]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job


root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje1/"
output_dir = output_path + "eje1/"


def map(key, values, context):
    words = values.split()
    for w in words:
        context.write(w[0], 1)
        
def freduce(key, values, context):
    c=0
    for v in values:
        c=c+1
    context.write(key, c)
    
def fShuffleCmp(aKey, anotherKey):
     if (aKey.lower() == anotherKey.lower()):
         return 0
     elif (aKey.lower() < anotherKey.lower()):
         return -1
     else: 
         return 1

job = Job(input_dir,output_dir,map,freduce)
job.setShuffleCmp(fShuffleCmp)
job.waitForCompletion()


True

### b

Se van a ejecutar tantos reducers como caracteres (en lower() indiferente si es "A" o "a") encuentre el map. No termino de entenderla, supongo que se podrían crear claves intermedias que hagan un random entre 1 y 2 pero luego los cmp nose como afectarían ***preguntar!***

## Ejercicio 2

Es muy util para tareas relacionadas a conteos, agregaciones, operaciones tipo base de datos, frecuencias. Con los mappers se debe prestar atención principalmente a que las claves intermedias emitidas sea coherentes con la lógica implementada en los diferentes cmp.

## Ejercicio 3

En principio el combiner no ayuda en nada, inclusive puede generar problemas, mi entendimiento es el siguiente, un combiner trabaja con la salida de los mappers, en este caso tenemos dos mappers, combinar las diferentes salidas puede mezclar info, tal vez podría servir para las cajas de ahorro, pero que es exactamente lo que combinaríamos, al fin y al cabo agrupar los valores de un cliente en una lista no tiene mucho sentido, porque ya es lo que le llega al reducer.

## Ejercicio 4

Queremos determinar la mediana de una serie de valores, en principio la tarea map lo que tiene que hacer es extraer el valor del data-set website y devolverlo, tendríamos que tener un solo reducer que pueda ver toda la lista en principio, pero seria interesante que el sort ya le de la lista ordenada de menor a mayor, lo que podemos hacer es emitir claves estilo 1x donde x es el numero que nos interesa, entonces en el shuffle simplemente miramos el 1, y luego en el sort ya si las ordenamos como se debe, luego miramos los value en el propio reducer, luego esta la lógica para determinar la mediana. Para el calcula de la ventana voy a replicar un lista en el reduce y listo. Si bien esto funciona, en una situación real no seria aplicable el reduce, lo que uno debería hacer es un primer job que cuente cuantos valores hay, y un segundo que efectivamente determine la mediana, el primero puedo setear un parametro del segundo.

In [4]:
import datetime,math,re
import random
import sys
sys.path.append('../..')
from MRE import Job


root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje2/"
output_dir = output_path + "eje2/"

def fmap(key, value, context):
    data = value.split('\t')
    time = data[1]
    context.write((1, time), time)

def fShuffleCmp(aKey, anotherKey):
    if (aKey[0] == anotherKey[0]):
        return 0
    elif (aKey[0] < anotherKey[0]):
        return -1
    else:
        return 1

def fSortCmp(aKey, anotherKey):
    if (aKey[1] == anotherKey[1]):
        return 0
    elif (aKey[1] < anotherKey[1]):
        return -1
    else:
        return 1

def fred(key, value, context):
    list_values = list(value)
    n = len(list_values)
    if n % 2 == 0:
        mediana = (list_values[n//2 - 1] + list_values[n//2]) / 2
    else:
        mediana = list_values[n//2]
    
    context.write("mediana", mediana)


job = Job(input_dir, output_dir, fmap, fred)
job.setShuffleCmp(fShuffleCmp)
job.setSortCmp(fSortCmp)
job.waitForCompletion()

True

## Ejercicio 5
Vamos a implementar dos mappers, uno se encarga de hacer lo mismo que hacia antes el otro simplemente lee los valores de las variables y los pasa a los reducers. Tenemos que modificar el shuffle y el sort para poder recibir primero todos los valores de las variables y luego realizar el calculo, ademas recibirlas en orden asi es mas fácil acceder a los valores.
Sobre este sistema debemos implementar un algoritmo que reciba la salida del reducer, osea los n valores de las variables y los replique n veces usando las diferentes claves.

In [6]:
import datetime,math,re
import random
import sys
sys.path.append('../..')
from MRE import Job


root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir1 = input_path + "eje5a/"
output_dir = output_path + "eje5/"
input_dir2 = input_path + "eje5b/"

def repeat_variables(input_path, output_path): 
    dict = {}
    with open(output_path,'r') as file:
        for line in file:
            data = line.split('\t')
            dict[data[0]] = data[1]
    
    with open(input_path, "w") as file: 
        for variable, value in dict.items(): 
            for variable2, value2 in dict.items():
                file.write(f'{variable}\t{variable2}\t{value2}')


def fmap1(key, value, context):
    data = value.split('\t')
    var = data[0]
    var_value = data[1]
    context.write((2, key), (2, var, var_value))
    
def fmap2(key, value, context):
    data = value.split('\t')
    var = data[0]
    var_value = data[1]
    context.write((1, key), (1, var, var_value))

def fShuffleCmp(actKey, otherKey):
    if (actKey[1] == otherKey[1]):
        return 0
    elif (actKey[1] < otherKey[1]):
        return -1
    else:
        return 1

def fSortCmp(actKey, otherKey): 
    if (actKey[0] == otherKey[0]):
        return 0
    elif (actKey[0] == 1):
        return -1
    else:
        return 1
    
    

def fred(key, value, context):
    res = 0
    var_value = [1] * 16
    for v in value:
        if (v[0] == 1):
            var_value[int(v[1].replace("var", ""))] = float(v[2])
        else:
            coef = v[1]
            coef_value = v[2]
            if (coef == "TI"):
                idx = 0
            else: 
                idx = int(coef.replace("var", ""))
            res += var_value[idx] * float(coef_value)
    context.write(key[1],res)

def read_unknowns ():
    output_file = output_dir + "output.txt"
    unknowns = [1] * 17  # Inicializa con 1 en todos los elementos
    with open(output_file, "r", encoding="utf-8") as f:
        for line in f:
            var, value = line.strip().split('\t')
            idx = int(var.replace("var", ""))
            unknowns[idx] = float(value)

    unknowns[0] = 1
    return unknowns

def calc_error (unknowns0, unknowns1):
    res = 0
    for i in range(15):
        res += (unknowns1[i+1] - unknowns0[i + 1]) ** 2
    return res
    
job1 = Job(input_dir1,output_dir,fmap1,fred)
job1.addInputPath(input_dir2, fmap2)
job1.setShuffleCmp(fShuffleCmp)
job1.setSortCmp(fSortCmp)
incog = {"unknowns": [1,-37,8,-5,4,5,6,7,1,2,5,2,2,1,3,4]}

error = 0.1
dif  = 1
i = 0
while(dif >= error):
    i += 1
    print(i)
    repeat_variables(input_dir2 + "input.txt",output_dir + "output.txt")
    job1.waitForCompletion()
    unknowns = read_unknowns()
    dif = calc_error(incog["unknowns"], unknowns)
    incog["unknowns"] = unknowns
print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
71


## Ejercicio 6

